In [1]:
import math
import pandas as pd
import numpy as np
import csv
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.style.use('seaborn')

#Statistical test
import scipy.stats as stats
from scipy.stats import chi2_contingency

#one_Hot_encoder
from sklearn import preprocessing

#classifiers comparisson
from sklearn.model_selection import train_test_split
from sklearn.utils import resample # Sampling for balanced classes

# Estandarización/Normalización a Z 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

#clasificadores
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import f1_score, roc_auc_score
from sklearn import linear_model # Regression Model

In [2]:
def entropy_y(y_values):
    fr_tab=pd.crosstab(index=y_values, columns="count")
    frecs=fr_tab/fr_tab.sum()
    return (sum([-(prop*math.log2(prop)) for prop in frecs["count"]]))

def entropy_xy(true_condition,reactivo):
    vars_df=pd.DataFrame({"true_condition":true_condition,"reactivo":reactivo})
    return(sum([(entropy_y(vars_df["true_condition"][vars_df["reactivo"]==resp])*
                 sum(vars_df["reactivo"]==resp)/len(vars_df["reactivo"])) 
                for resp in set(vars_df["reactivo"])]))

def info_gain(true_condition,reactivo):
    return (entropy_y(true_condition)-entropy_xy(true_condition,reactivo))

def subs_balancer(x_df, y_df, small_class_label, big_class_label):
    sml_class_idx=list(y_df.loc[y_df==small_class_label].index)
    big_class_idx=list(y_df.loc[y_df==big_class_label].index)
    subsample_big_class_idx=resample(big_class_idx,
                                replace=False, # sample without replacement
                                n_samples=len(sml_class_idx)) # to match minority class
    sml_class_idx.extend(subsample_big_class_idx)

    return x_df.loc[sml_class_idx,],y_df[sml_class_idx]

def subs_balancer_indexer(y_df, small_class_label, big_class_label):
    sml_class_idx=list(y_df.loc[y_df==small_class_label].index)
    big_class_idx=list(y_df.loc[y_df==big_class_label].index)
    subsample_big_class_idx=resample(big_class_idx,
                                replace=False, # sample without replacement
                                n_samples=len(sml_class_idx)) # to match minority class
    sml_class_idx.extend(subsample_big_class_idx)
    return sml_class_idx

In [ ]:
h = .02  # step size in the mesh

names = [
    "Logistic Regression",
    "Nearest Neighbors",
    "Linear SVM", 
#     "RBF SVM", 
#     "Gaussian Process",
    "Decision Tree", 
    "Random Forest", 
    "Neural Net", 
    "AdaBoost",
    "Naive Bayes", 
    "QDA"]

classifiers = [
    linear_model.LogisticRegression(n_jobs=-1),
    KNeighborsClassifier(5, n_jobs=-1),
    SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
    DecisionTreeClassifier(max_depth=10),
    RandomForestClassifier(max_depth=10, n_estimators=100,n_jobs=-1),
    MLPClassifier(alpha=0.01, learning_rate="adaptive", max_iter=2000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

# Ejercicio 1

In [2]:
X_train = pd.read_csv("train.csv", sep=";", encoding='latin-1')
train.head()

,id,Fecha.Expedicion,Cancelacion,Gestionable,Retencion,TIPO,ANO_MES,Target,Fecha.Proceso,Disponible.Avances,...,Saldos.Mes.Ant,Pagos.Mes.Ant,Vtas.Mes.Ant,Edad.Mora,Limite.Cupo,Pago.del.Mes,Pago.Minimo,Vr.Mora,Vr.Cuota.Manejo,Saldo
0,1,4/11/2006,NaN,NaN,NaN,NaN,NaN,0,1/04/2018,1 050 000.00,...,0,0,0,0,2 625 000.00,0,0,0,0,0
1,2,17/08/2007,NaN,NaN,NaN,NaN,NaN,0,1/04/2018,1 180 000.00,...,0,0,0,0,2 950 000.00,0,0,0,0,0
2,3,8/05/2008,NaN,NaN,NaN,NaN,NaN,0,1/04/2018,150 000.00,...,0,0,0,0,500 000.00,0,0,0,0,0
3,4,21/12/2006,NaN,NaN,NaN,NaN,NaN,0,1/04/2018,680 000.00,...,0,0,0,0,1 700 000.00,0,0,0,0,0
4,5,13/09/2007,NaN,NaN,NaN,NaN,NaN,0,1/04/2018,3 307 500.00,...,0,0,0,0,6 615 000.00,0,0,0,0,0


In [3]:
test = pd.read_csv("test.csv", sep=";", encoding='latin-1')
test.head()

,id,Fecha.Expedicion,Cancelacion,Gestionable,TIPO,ANO_MES,Fecha.Proceso,Disponible.Avances,Limite.Avances,Total.Intereses,Saldos.Mes.Ant,Pagos.Mes.Ant,Vtas.Mes.Ant,Edad.Mora,Limite.Cupo,Pago.del.Mes,Pago.Minimo,Vr.Mora,Vr.Cuota.Manejo,Saldo
0,50002,14/06/2007,NaN,NaN,NaN,NaN,1/04/2018,150 000.00,150 000.00,0,0,0,0,0,500 000.00,0,0,0,0,0
1,50003,18/09/2007,NaN,NaN,NaN,NaN,1/04/2018,150 000.00,150 000.00,0,0,0,0,0,500 000.00,0,0,0,0,0
2,50004,12/04/2007,NaN,NaN,NaN,NaN,1/04/2018,540 000.00,540 000.00,0,0,0,0,0,1 350 000.00,0,0,0,0,0
3,50005,17/11/2006,NaN,NaN,NaN,NaN,1/04/2018,252 000.00,252 000.00,0,0,0,0,0,840 000.00,0,0,0,0,0
4,50006,31/01/2007,NaN,NaN,NaN,NaN,1/04/2018,760 000.00,760 000.00,0,0,0,0,0,1 900 000.00,0,0,0,0,0


In [4]:
#pd.isna(train['ANO_MES'])
my_tab = pd.crosstab(1*(train['ANO_MES']>0),train['Target'])
#train['Target']
my_tab

Target,0,1
ANO_MES,,
0,48589,0
1,0,1412


In [5]:
roc_auc_score(train['Target'],1*(train['ANO_MES']>0))

1.0

In [6]:
predict=1*(test['ANO_MES']>0)

In [7]:
prediction=pd.DataFrame({'id':test['id'], 'predict':predict})

In [8]:
prediction.to_csv("prediction.csv", sep=";", encoding="latin-1")

# Ejercicio 2

In [9]:
def validador(orig_fila, orig_columna, dest_fila, dest_columna):
    if (((orig_fila>=1)&(orig_fila<=8))&((orig_columna>=1)&(orig_columna<=8))&((dest_fila>=1)&(dest_fila<=8))&((dest_columna>=1)&(dest_columna<=8))):
        if((orig_fila==dest_fila)&(orig_columna==dest_columna)):
            print ("ERROR!: Misma posición")
            return False
        else:
            return True
    else: 
        print ("ERROR!: Coordenadas fuera de los límites")
        return False

def distancias (orig_fila, orig_columna, dest_fila, dest_columna):
    h_dist=orig_fila-dest_fila
    v_dist=orig_columna-dest_columna
    return([h_dist,v_dist])

In [10]:
def mov_caballo(orig_fila, orig_columna, dest_fila, dest_columna):
    if validador(orig_fila, orig_columna, dest_fila, dest_columna):
        dists=distancias (orig_fila, orig_columna, dest_fila, dest_columna)
        dist_euclid=math.sqrt(dists[0]**2+dists[1]**2)
        if (dist_euclid==math.sqrt(5)):
            print("Movimiento válido")
            return 1
        else:
            print("Movimiento Inválido")
            return 0
    else:
        return 0

In [11]:
def mov_reina(orig_fila, orig_columna, dest_fila, dest_columna):
    if validador(orig_fila, orig_columna, dest_fila, dest_columna):
        dists=distancias (orig_fila, orig_columna, dest_fila, dest_columna)
        if ((dists[0]==0)or(dists[1]==0)or(abs(dists[0])-abs(dists[1])==0)):
            print("Movimiento válido")
            return 1
        else:
            print("Movimiento Inválido")
            return 0
    else:
        return 0            

In [12]:
p_inic=[6,2]

In [13]:
[[[p_inic[0],p_inic[1],i,j],mov_caballo(p_inic[0],p_inic[1],i,j)] for i in (range(1,9))for j in (range(1,9))]
#print(sum([mov_caballo(p_inic[0],p_inic[1],i,j) for i in (range(1,9))for j in (range(1,9))]))

Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento válido
Movimiento Inválido
Movimiento válido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento válido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
ERROR!: Misma posición
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Mov

[[[6, 2, 1, 1], 0],
 [[6, 2, 1, 2], 0],
 [[6, 2, 1, 3], 0],
 [[6, 2, 1, 4], 0],
 [[6, 2, 1, 5], 0],
 [[6, 2, 1, 6], 0],
 [[6, 2, 1, 7], 0],
 [[6, 2, 1, 8], 0],
 [[6, 2, 2, 1], 0],
 [[6, 2, 2, 2], 0],
 [[6, 2, 2, 3], 0],
 [[6, 2, 2, 4], 0],
 [[6, 2, 2, 5], 0],
 [[6, 2, 2, 6], 0],
 [[6, 2, 2, 7], 0],
 [[6, 2, 2, 8], 0],
 [[6, 2, 3, 1], 0],
 [[6, 2, 3, 2], 0],
 [[6, 2, 3, 3], 0],
 [[6, 2, 3, 4], 0],
 [[6, 2, 3, 5], 0],
 [[6, 2, 3, 6], 0],
 [[6, 2, 3, 7], 0],
 [[6, 2, 3, 8], 0],
 [[6, 2, 4, 1], 1],
 [[6, 2, 4, 2], 0],
 [[6, 2, 4, 3], 1],
 [[6, 2, 4, 4], 0],
 [[6, 2, 4, 5], 0],
 [[6, 2, 4, 6], 0],
 [[6, 2, 4, 7], 0],
 [[6, 2, 4, 8], 0],
 [[6, 2, 5, 1], 0],
 [[6, 2, 5, 2], 0],
 [[6, 2, 5, 3], 0],
 [[6, 2, 5, 4], 1],
 [[6, 2, 5, 5], 0],
 [[6, 2, 5, 6], 0],
 [[6, 2, 5, 7], 0],
 [[6, 2, 5, 8], 0],
 [[6, 2, 6, 1], 0],
 [[6, 2, 6, 2], 0],
 [[6, 2, 6, 3], 0],
 [[6, 2, 6, 4], 0],
 [[6, 2, 6, 5], 0],
 [[6, 2, 6, 6], 0],
 [[6, 2, 6, 7], 0],
 [[6, 2, 6, 8], 0],
 [[6, 2, 7, 1], 0],
 [[6, 2, 7, 2], 0],


In [14]:
[[[p_inic[0],p_inic[1],i,j],mov_reina(p_inic[0],p_inic[1],i,j)] for i in (range(1,9))for j in (range(1,9))]
#print(sum([mov_reina(p_inic[0],p_inic[1],i,j) for i in (range(1,9))for j in (range(1,9))]))

Movimiento Inválido
Movimiento válido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento válido
Movimiento Inválido
Movimiento Inválido
Movimiento válido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento válido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento válido
Movimiento Inválido
Movimiento Inválido
Movimiento válido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento válido
Movimiento Inválido
Movimiento válido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento válido
Movimiento válido
Movimiento válido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento Inválido
Movimiento válido
ERROR!: Misma posición
Movimiento válido
Movimiento válido
Movimiento válido
Movimiento válido
Movimiento válido
Movimiento válido
Movimiento válido
Movimiento válido
Movimiento válido
Movimiento Inválido

[[[6, 2, 1, 1], 0],
 [[6, 2, 1, 2], 1],
 [[6, 2, 1, 3], 0],
 [[6, 2, 1, 4], 0],
 [[6, 2, 1, 5], 0],
 [[6, 2, 1, 6], 0],
 [[6, 2, 1, 7], 1],
 [[6, 2, 1, 8], 0],
 [[6, 2, 2, 1], 0],
 [[6, 2, 2, 2], 1],
 [[6, 2, 2, 3], 0],
 [[6, 2, 2, 4], 0],
 [[6, 2, 2, 5], 0],
 [[6, 2, 2, 6], 1],
 [[6, 2, 2, 7], 0],
 [[6, 2, 2, 8], 0],
 [[6, 2, 3, 1], 0],
 [[6, 2, 3, 2], 1],
 [[6, 2, 3, 3], 0],
 [[6, 2, 3, 4], 0],
 [[6, 2, 3, 5], 1],
 [[6, 2, 3, 6], 0],
 [[6, 2, 3, 7], 0],
 [[6, 2, 3, 8], 0],
 [[6, 2, 4, 1], 0],
 [[6, 2, 4, 2], 1],
 [[6, 2, 4, 3], 0],
 [[6, 2, 4, 4], 1],
 [[6, 2, 4, 5], 0],
 [[6, 2, 4, 6], 0],
 [[6, 2, 4, 7], 0],
 [[6, 2, 4, 8], 0],
 [[6, 2, 5, 1], 1],
 [[6, 2, 5, 2], 1],
 [[6, 2, 5, 3], 1],
 [[6, 2, 5, 4], 0],
 [[6, 2, 5, 5], 0],
 [[6, 2, 5, 6], 0],
 [[6, 2, 5, 7], 0],
 [[6, 2, 5, 8], 0],
 [[6, 2, 6, 1], 1],
 [[6, 2, 6, 2], 0],
 [[6, 2, 6, 3], 1],
 [[6, 2, 6, 4], 1],
 [[6, 2, 6, 5], 1],
 [[6, 2, 6, 6], 1],
 [[6, 2, 6, 7], 1],
 [[6, 2, 6, 8], 1],
 [[6, 2, 7, 1], 1],
 [[6, 2, 7, 2], 1],
